In [ ]:
import os
import json
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

folder_path = '/content/drive/MyDrive/doctors'
doctors_data = load_folder(folder_path)

In [ ]:
def process_single_file(file_path):
    """
    Обрабатывает один JSON или JSONL файл и возвращает список словарей
    """
    data_list = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f: # открываем файл для чтения с кодировкой UTF-8
            if file_path.endswith('.jsonl'):
                for line in f: # жля JSONL файлов читаем файл построчно
                    line = line.strip()
                    if line:
                        data = json.loads(line) # преобразуем строку JSON в объект Python
                        if isinstance(data, dict): # проверяем является ли data словарем
                            data_list.append(data) # если да, то добавляем в список
                        elif isinstance(data, list):
                            data_list.extend(data) # если нет, то мы как бы распаковываем и добавляем в спиок (например, если у нас было два словаря в списке)
            else: # аналогично, но без построчной обработки, так как это JSON
                data = json.load(f)
                if isinstance(data, dict):
                    data_list.append(data)
                elif isinstance(data, list):
                    data_list.extend(data)
    except Exception as e:
        print(f"Ошибка в файле {file_path}: {str(e)}")
    return data_list

In [ ]:
def load_folder(folder_path):
    all_data = []

    for file in os.listdir(folder_path): # получаем список всех файлов и папок в указанной директории
        if file.endswith(('.json', '.jsonl')):
            file_path = os.path.join(folder_path, file) # создаем полный путь к файлу
            all_data.extend(process_single_file(file_path)) # добавляем все обработанные файлы в общий список (extend тк "распаковываем" список и добавляем его элементы)

    return all_data

In [ ]:
def prepare_csv_data(doctors_data):
    csv_data = []

    for doctor in doctors_data:

        # Основная информация
        row = {
            'name': doctor.get('name', None),
            'speciality': doctor.get('speciality', None),
            'experience': doctor.get('experience', None),
            'rating': doctor.get('rating', None),
            'review_count': doctor.get('review_count', 0),
            'price': doctor.get('price', None),
            'is_kids': doctor.get('is_kids', False),
            'is_adults': doctor.get('is_adults', False),
            'link': doctor.get('link', None),
        }

        if doctor.get('clinics', None) is None:
            row['clinics_count'] = 0
            for i in range(3):
                row[f'clinic_{i+1}_name'] = None
                row[f'clinic_{i+1}_address'] = None
                row[f'clinic_{i+1}_metro'] = None

        else:
            row['clinics_count'] = len(doctor.get('clinics', []))
            # Информация о клиниках
            clinics = doctor.get('clinics', [])
            for i, clinic in enumerate(clinics[:3]):  # максимум 3 клиники
                row[f'clinic_{i+1}_name'] = clinic.get('name', None)
                row[f'clinic_{i+1}_address'] = clinic.get('address', None)
                row[f'clinic_{i+1}_metro'] = clinic.get('metro', None)

            # Заполняем пустые клиники
            for i in range(len(clinics), 3):
                row[f'clinic_{i+1}_name'] = None
                row[f'clinic_{i+1}_address'] = None
                row[f'clinic_{i+1}_metro'] = None

        csv_data.append(row)

    return csv_data

In [ ]:
def prepare_review_csv_data(doctors_data):
    csv_data = []

    for doctor in doctors_data:
      if doctor.get('reviews', None) is None:
        row = {
            'doctor_name': doctor.get('name', None),
            'doctor_link': doctor.get('link', None),
            'rate': None,
            'date': None,
            'comment': None,
            'clinic': None
        }
        csv_data.append(row)

      else:
        doctor_name = doctor.get('name', None)
        doctor_link = doctor.get('link', None)
        for review in doctor.get('reviews'):
          row = {
            'doctor_name': doctor_name,
            'doctor_link': doctor_link,
            'rate': review.get('rate', None),
            'date': review.get('date', None),
            'comment': review.get('comment', None),
            'clinic': review.get('clinic', None)
          }
          csv_data.append(row)

    return csv_data

In [ ]:
def save_to_dataset(csv_data, filename, folder_path):
    csv_filename = f"{filename}.csv"
    df = pd.DataFrame(csv_data)
    file_path = os.path.join(folder_path, csv_filename)
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    return df

In [ ]:
folder_path = '/content/drive/MyDrive/doctors'
doctors_data = load_folder(folder_path)
df = save_to_dataset(prepare_csv_data(doctors_data), 'doctors', folder_path)
df_reviews = save_to_dataset(prepare_review_csv_data(doctors_data), 'reviews', folder_path)

In [ ]:
df

In [ ]:
df_reviews